In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [13]:
print("pandas: ", pd.__version__)
print("seaborn: ",sns.__version__)

pandas:  2.1.0
seaborn:  0.12.2


# Ler microdados do ENEM

## Selecionar apenas uma amostra aleatória de 1% dos alunos
Se a gente lê o arquivo de 3GB inteiro, vai dar problema. Aqui tem uma receita para ler somente x%: https://www.kaggle.com/questions-and-answers/53925 

In [14]:
def skip(i,fraction=0.01):
    if i == 0:
        return False  # precisamos manter o row=0 para ter o header
    else:
        return random.random() > fraction

[skip(i) for i in range(10)]

[False, True, True, True, True, True, True, True, True, True]

A título de exemplo, vamos carregar os dados de 2010.

In [15]:
%time df = pd.read_csv('../inep/enem/2010/DADOS/MICRODADOS_ENEM_2010.csv',skiprows = skip, encoding='latin1',sep=";")

CPU times: user 18.8 s, sys: 1.33 s, total: 20.1 s
Wall time: 20.1 s


In [16]:
df

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57
0,200000000068,2010,12,M,1,1,1,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200000000111,2010,5,M,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200000000231,2010,4,F,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200000000356,2010,11,F,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200000000439,2010,4,F,0,1,2,1.0,2304400.0,FORTALEZA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45824,200004611062,2010,15,F,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45825,200004611079,2010,14,F,1,2,1,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45826,200004611170,2010,6,F,0,4,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45827,200004611412,2010,14,F,0,2,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


De fato, só vem 50 mil linhas (os dados originais devem conter da ordem de 3-6 milhões de participantes)

Em seguinda, vamos filtrar estes dados pela presença na prova, se é treineiro e estar concluindo o EM.

In [17]:
df["TP_PRESENCA_CN"].value_counts()

TP_PRESENCA_CN
1.0    33472
0.0    12306
2.0       50
Name: count, dtype: int64

In [18]:
df['TP_ST_CONCLUSAO'].value_counts()

TP_ST_CONCLUSAO
1    26858
2    13680
3     5291
Name: count, dtype: int64

## Filtrar os dados por presença, treineiro, EM e notas não-zero

In [19]:
def filterdf(df):
    df = df[df["TP_PRESENCA_CN"] == 1]  # presente nas 4 provas
    df = df[df["TP_PRESENCA_CH"] == 1]
    df = df[df["TP_PRESENCA_LC"] == 1]
    df = df[df["TP_PRESENCA_MT"] == 1]
    if "IN_TREINEIRO" in df.columns:
        df = df[(df["IN_TREINEIRO"] == 0) | (df["IN_TREINEIRO"].isna())]  # não é treineiro ou não existe 
    df = df[df["TP_ST_CONCLUSAO"].isin([1, 2])]  # afirma que concluiu ou vai concluir EM em 2019
    # não queremos as notas == 0
    df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0")
    df.dropna(subset = ['TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT']) # achamos linhas com NaN para estas colunas!
    return df


def load_data(ano,frac):
    'carrega dados com determinado ano e fração'
    print(f"processando {ano} com fraçao = {frac*100:.0f}%",ano, frac)
    def skip(i,fraction=frac):
        if i == 0:
            return False  # precisamos manter o row=0 para ter o header
        else:
            return random.random() > fraction
   
    if ano == 2016:
        caminho = f'../inep/enem/{ano}/DADOS/microdados_enem_{ano}.csv'
    else:
        caminho = f'../inep/enem/{ano}/DADOS/MICRODADOS_ENEM_{ano}.csv'
    #print(caminho)
    df = pd.read_csv(caminho,skiprows = skip, encoding='latin1',sep=";")
    return df
    
def filter_data(df):
    print("filter data...")
    return filterdf(df)

def nome_do_arquivo(ano,frac):
    frac = frac*100
    fn = f'data/enem_{frac:.0f}_{ano}.csv'
    return fn

df = load_data(2014,0.01)
print(len(df))
df = filter_data(df)
print(len(df))


processando 2014 com fraçao = 1% 2014 0.01
87375
filter data...
45691


## Aqui faremos o trabalho de verdade
Isso deve levar alguns minutos.

In [ ]:
frac = 0.01
for ano in range(2009,2023):
    df = load_data(ano,frac)
    df = filter_data(df)
    fn = nome_do_arquivo(ano,frac)
    #print(fn+"...\n")
    df.to_csv(fn,index=False)

import inspect
filter_conditions = inspect.getsource(filterdf)
with open('data/filters.txt','w') as f:
    f.write(filter_conditions)

processando 2009 com fraçao = 1% 2009 0.01
filter data...
processando 2010 com fraçao = 1% 2010 0.01
filter data...
processando 2011 com fraçao = 1% 2011 0.01
filter data...
processando 2012 com fraçao = 1% 2012 0.01
filter data...
processando 2013 com fraçao = 1% 2013 0.01
filter data...
processando 2014 com fraçao = 1% 2014 0.01
filter data...
processando 2015 com fraçao = 1% 2015 0.01
filter data...
processando 2016 com fraçao = 1% 2016 0.01


# Preparar os dados para análise TRI
Precisamos converter as respostas e o gabarito no microdados do INEP para um formato que os pacotes de TRI conseguem ler.

## Funções para extrair acertos
A maioria das análises TRI vão precisar de uma "dicomitização" das respostas. A estrutura de dados que vamos precisar é uma matriz com colunas itens e linhas pessoas com valores 0 e 1 para erros e acertos respectivamente.

Vamos tentar criar umas funções que crie este estrutura de dados.


Temos algumas informações sobre a estrutura dos microdados:
* não vamos usar as primeiras 10 itens da prova LC (Linguágens e Códigos). Desde 2011 (?) estas posições são usadas para as questões de língua estrangeira (espanhol ou inglês). Acredito que somente 40 questões de LC são usados para criar a escala IRT desta prova.
* O INEP divulga, desde o final de 2022, os parámetros IRT (discriminação, dificuldade e c). Além disso, dizem quais itens foram eliminados ("pelo IRT"), porque aparentemente atrapalharam a convergência da estimação do modelo IRT 3PL que estão usando. Vamos ter que tirar estes itens também.  

In [ ]:
# Assumimos aqui que as planilhas com informações sobre os itens do INEP já foram convertidos para utf-8 e ficam no diretório DADOS
def load_amostra(ano,perc):
    df = pd.read_csv(f'data/enem_{perc}_{ano}.csv',dtype={'CO_PROVA_LC':int,'CO_PROVA_CN':int,'CO_PROVA_CH':int,'CO_PROVA_MT':int})
    item_info = pd.read_csv(f'../inep/enem/{ano}/DADOS/ITENS_PROVA_{ano}-utf8.csv',sep='\;',engine='python')
    item_info.dropna(subset='CO_ITEM',inplace=True)
    item_info['CO_ITEM'] = item_info['CO_ITEM'].astype(int)
                     
    #print(list(item_info.columns),'\n')
    #print(list(df.columns))
    return df, item_info

df, itens = load_amostra(2013,1)


In [ ]:
df[:2]

In [ ]:
itens[:2]

In [ ]:
def to_acertos(s):
    'resp e gab são strings, retorna uma lista'
    resp = s.iloc[0]
    gab = s.iloc[1]
    return [r == g for r,g in zip(resp,gab)]

def acertos_df(df,exame,as_int=True):
    'Retorna dataframe com acertos. Colunas 1-45 = itens, Linhas = idx do df'
    resp_col = 'TX_RESPOSTAS_' + exame
    gab_col = 'TX_GABARITO_' + exame
    adf = df[[resp_col,gab_col]].apply(to_acertos,axis=1,result_type = "expand")
    adf['acertos'] = adf.sum(axis=1)
    adf['caderno'] = df.loc[:,"CO_PROVA_" + exame]
    if as_int:
        adf = adf.astype("int")
    adf['nota_inep'] = df.loc[:,"NU_NOTA_" + exame]
    return adf

#item_info = pd.read_csv('data/ITENS_PROVA_2019-utf8.csv')

def reorder_remove_cols(ac,item_info):
    'takes a df with acertos, cuts it up and renames the columns for each caderno and stitches it back again.'
    gb = ac.groupby("caderno")
    
    itemgroups = item_info.groupby("CO_PROVA")
    cadernos = ac['caderno'].value_counts().index[:4] #only the 4 most used cadernos
    groups = []
    for caderno in cadernos:
        #print(caderno)
        itemnames = itemgroups.get_group(caderno).sort_values("CO_POSICAO")["CO_ITEM"].values
        colmap = {i:j for i,j in zip(range(len(itemnames)),itemnames)}
        group = gb.get_group(caderno).rename(colmap,axis=1)
        groups.append(group)
    ac = pd.concat(groups)
    removed_items = item_info.query("IN_ITEM_ABAN == 1")["CO_ITEM"].unique()
    ac = ac.drop(removed_items,axis=1,errors='ignore')
    return ac


    

In [ ]:
perc = 1
for ano in range(2014,2023):
    df,itens = load_amostra(ano,perc)
    for area in ['CH', 'CN', 'MT']: # LC tem o problema de 50 itens no gabarito
        ac = acertos_df(df,area)
        ac = reorder_remove_cols(ac,itens)
        ac.to_csv(f'data/ac_{perc}_{ano}_{area}.csv',index=False)

